In [5]:
import pandas as pd

# Customer data inputs
customer_data = [
    {
        'Customer ID': "CUST9997",
        'Average Order Value': 58.59,
        'Purchase Frequency': 19,
        'Days Since Last Purchase': 335,
        'Total Orders': 24,
        'Customer Tenure': 4,
        'Engagement Level': 42,
        'Retention Risk Profile': 1
    },
    {
        'Customer ID': "CUST9998",
        'Average Order Value': 60.00,
        'Purchase Frequency': 19,
        'Days Since Last Purchase': 335,
        'Total Orders': 40,
        'Customer Tenure': 4,
        'Engagement Level': 42,
        'Retention Risk Profile': 1
    }
]

# Transaction data inputs
transaction_data = [
    {
        'Customer ID': "CUST9998",
        'Transaction Amount': 789.13,
        'Transaction Date': "2024-05-30T00:05:54.003+00:00",
        'CLV': 0.8631578947368421
    },
    {
        'Customer ID': "CUST9997",
        'Transaction Amount': 89.13,
        'Transaction Date': "2024-05-30T00:05:54.003+00:00",
        'CLV': 0.8631578947368421
    }
]

# Create DataFrames
customer_df = pd.DataFrame(customer_data)
transaction_df = pd.DataFrame(transaction_data)

# Write to CSV files
customer_df.to_csv('customer.csv', index=False)
transaction_df.to_csv('transaction.csv', index=False)

print("CSV files created: 'customer_data.csv' and 'transaction_data.csv'")


CSV files created: 'customer_data.csv' and 'transaction_data.csv'


In [6]:
import pandas as pd

# Read the customer and transaction data from their respective CSV files
customer_df = pd.read_csv('customer.csv')
transaction_df = pd.read_csv('transaction.csv')

# Add a new column to distinguish the types of data
customer_df['Type'] = 'Customer'
transaction_df['Type'] = 'Transaction'

# Concatenate the DataFrames
combined_df = pd.concat([customer_df, transaction_df], ignore_index=True)

# Write the combined DataFrame to a new CSV file
combined_df.to_csv('combine.csv', index=False)

print("Combined CSV file created: 'combine.csv'")


Combined CSV file created: 'combine.csv'


In [ ]:
import pandas as pd
from pymongo import MongoClient

# Step 1: Connect to MongoDB
connection_string = ''

try:
    client = MongoClient(connection_string)
    
    # Access the CRM database
    db = client['CRM']
    
    # Access the collections
    customer_collection = db['CustomerData']
    transaction_collection = db['TransactionData']

    # Step 2: Read the combined data from the CSV file
    combined_df = pd.read_csv('combine.csv')

    # Step 3: Separate the data based on the 'Type' column
    customer_data = combined_df[combined_df['Type'] == 'Customer'].drop(columns=['Type'])
    transaction_data = combined_df[combined_df['Type'] == 'Transaction'].drop(columns=['Type'])

    # Step 4: Insert customer data into MongoDB
    customer_records = customer_data.to_dict(orient='records')
    if customer_records:
        result = customer_collection.insert_many(customer_records)
        print(f"Inserted {len(result.inserted_ids)} records into CustomerData collection.")
        print("Inserted IDs:", result.inserted_ids)  # Print inserted IDs for verification

    # Step 5: Insert transaction data into MongoDB
    transaction_records = transaction_data.to_dict(orient='records')
    if transaction_records:
        result = transaction_collection.insert_many(transaction_records)
        print(f"Inserted {len(result.inserted_ids)} records into TransactionData collection.")
        print("Inserted IDs:", result.inserted_ids)  # Print inserted IDs for verification

    # Optional: Fetch and print all data from each collection
    all_customer_data = customer_collection.find()
    print("\nCustomer Data:")
    for record in all_customer_data:
        print(record)

    all_transaction_data = transaction_collection.find()
    print("\nTransaction Data:")
    for record in all_transaction_data:
        print(record)

except Exception as e:
    print(f"An error occurred: {e}")


Inserted 2 records into CustomerData collection.
Inserted IDs: [ObjectId('671d732fca465bec5480affc'), ObjectId('671d732fca465bec5480affd')]
Inserted 2 records into TransactionData collection.
Inserted IDs: [ObjectId('671d7330ca465bec5480affe'), ObjectId('671d7330ca465bec5480afff')]

Customer Data:
{'_id': ObjectId('671d6beeca465bec5480afed'), 'Customer ID': 'CUST9997', 'Average Order Value': 58.59, 'Purchase Frequency': 19.0, 'Days Since Last Purchase': 335.0, 'Total Orders': 24.0, 'Customer Tenure': 4.0, 'Engagement Level': 42.0, 'Retention Risk Profile': 1.0, 'Transaction Amount': nan, 'Transaction Date': nan, 'CLV': nan}
{'_id': ObjectId('671d6beeca465bec5480afee'), 'Customer ID': 'CUST9998', 'Average Order Value': 60.0, 'Purchase Frequency': 19.0, 'Days Since Last Purchase': 335.0, 'Total Orders': 40.0, 'Customer Tenure': 4.0, 'Engagement Level': 42.0, 'Retention Risk Profile': 1.0, 'Transaction Amount': nan, 'Transaction Date': nan, 'CLV': nan}
{'_id': ObjectId('671d6e244d2e0e0f2b